In [2]:
using CSV, FileIO

In [3]:
data = load("../data/impvol_data.jld2")

Dict{String,Any} with 10 entries:
  "import_shares"             => [0.0 7.85e-7 … 0.000689 0.0004425; 7.42e-5 0.0…
  "pwt"                       => [0.824111 0.620073 … 0.747101 1.0]…
  "va"                        => [3271.5 1407.45 … 4209.16 41311.0]…
  "p_sectoral_data"           => [33.0109 64.7924 … 16.8926 36.7882]…
  "io_values"                 => [2.49202e5 2423.0 … 6491.5 0.0; 4183.8 56086.1…
  "total_output"              => [2.00728e6 8.11942e5 … 1.74308e6 66853.4]…
  "intermediate_input_shares" => [0.973154 0.0268455 … 0.528231 0.294862]…
  "trade_balance"             => [1184.95 -1239.14 … -406.84 1074.1]…
  "output_shares"             => [0.967079 0.032921 … 0.521898 0.318506]…
  "beta"                      => [0.230777 0.441 … 0.297884 0.370242]…

In [4]:
country_names = CSV.read("../experiments/baseline/output_table.csv")[:country_names]

25-element Array{Union{Missings.Missing, String},1}:
 "Australia"             
 "Austria"               
 "Belgium and Luxembourg"
 "Canada"                
 "China"                 
 "Colombia"              
 "Denmark"               
 "Finland"               
 "France"                
 "Germany"               
 "Greece"                
 "India"                 
 "Ireland"               
 "Italy"                 
 "Japan"                 
 "Mexico"                
 "Netherlands"           
 "Norway"                
 "Portugal"              
 "ROW"                   
 "South Korea"           
 "Spain"                 
 "Sweden"                
 "United Kingdom"        
 "United States"         

In [5]:
function US_price_index(data, experiment)
    p_sectoral = data["p_sectoral_data"][1,end,:,:]
    parameters = load("../experiments/$experiment/common_parameters.jld2")["parameters"]
    nu = parameters[:nu_njt][1,1,:,:]
    sigma = parameters[:sigma]
    return sum(nu .* p_sectoral .^ (1-sigma), 1) .^ (1/(1-sigma))
end

US_price_index (generic function with 1 method)

In [6]:
cpi = US_price_index(data, "baseline")

1×36 Array{Float64,2}:
 35.4369  38.2792  42.5554  46.6403  …  101.635  106.828  112.764  121.02

In [7]:
# to conform with data structures
us_cpi = reshape(cpi[end,:] ./ cpi[end,1], (1,1,1,36))

1×1×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 1.0

[:, :, 1, 2] =
 1.08021

[:, :, 1, 3] =
 1.20088

...

[:, :, 1, 34] =
 3.0146

[:, :, 1, 35] =
 3.18211

[:, :, 1, 36] =
 3.41508

In [8]:
dollar_price_index = data["pwt"] .* us_cpi

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 0.824111  0.620073  0.816447  1.0138  …  0.469671  1.04781  0.747101  1.0

[:, :, 1, 2] =
 1.13936  0.795024  0.995663  1.11022  …  0.59099  1.25141  0.825284  1.08021

[:, :, 1, 3] =
 1.52996  0.933944  1.14979  1.28988  …  0.71763  1.38363  0.944267  1.20088

...

[:, :, 1, 34] =
 2.9687  3.10978  3.24485  2.87797  …  2.75216  3.65456  3.34459  3.0146

[:, :, 1, 35] =
 3.09307  3.27387  3.43026  3.21288  …  2.95291  3.85804  3.56349  3.18211

[:, :, 1, 36] =
 3.69163  3.80254  3.96708  3.60814  …  3.46297  4.4728  4.11468  3.41508

In [9]:
real_GDP_data = sum(data["va"], 3) ./ dollar_price_index

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 64455.1  32858.7  40728.9  96150.9  …  37030.6  1.99307e5  1.12031e6

[:, :, 1, 2] =
 65982.8  31871.0  43135.0  1.0269e5  …  37735.3  2.06264e5  1.16062e6

[:, :, 1, 3] =
 62482.1  32631.5  43912.9  1.08415e5  …  39417.2  1.97532e5  1.13239e6

...

[:, :, 1, 34] =
 2.07894e5  88385.4  1.14103e5  3.67082e5  …  88693.0  6.07157e5  3.88509e6

[:, :, 1, 35] =
 2.14854e5  89878.8  114863.0  371576.0  …  90640.8  6.10731e5  3.89961e6

[:, :, 1, 36] =
 2.14192e5  89132.5  1.14673e5  3.69017e5  …  90536.3  6.08616e5  3.80577e6

In [10]:
include("../output.jl")
using ImpvolOutput
parameters = load("../experiments/baseline/common_parameters.jld2")["parameters"]
data_volatility = ImpvolOutput.calculate_volatilities(real_GDP_data, parameters, true)[:].^0.5

25-element Array{Float64,1}:
 0.0303417
 0.0142093
 0.0178478
 0.0139503
 0.0762514
 0.0293101
 0.0168866
 0.0209671
 0.0136236
 0.01504  
 0.0190854
 0.0307193
 0.0251652
 0.0138399
 0.0168999
 0.0275213
 0.0116397
 0.0253615
 0.0338734
 0.0401589
 0.0315266
 0.0137769
 0.0153384
 0.0149432
 0.0160794

In [11]:
function volatility(scenario)
    results = load("../experiments/$scenario/results.jld2")["results"]
    real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)
    return ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true)[:].^0.5
end

volatility (generic function with 1 method)

In [12]:
sigmas = [0.3 0.4 0.5 0.7 0.8 0.9 0.999 1.1 1.2 1.3 1.5 1.75 2.0]
correlations = zeros(sigmas)
no_china = [i for i in 1:parameters[:N] if i!=5]

i = 1
for sigma in sigmas
    model_volatility = volatility("CES/$sigma")
    correlations[i] = cor(data_volatility[no_china], model_volatility[no_china])
    i += 1
end

In [13]:
using Plots
plot(sigmas', correlations', label="", xlabel="Elasticity of substitution", ylabel="Correlation of volatilities in model and data (w/o China)", size=(800,500))
savefig("../output/Admissible-EOS.pdf")

In [14]:
data_volatility[no_china]

24-element Array{Float64,1}:
 0.0303417
 0.0142093
 0.0178478
 0.0139503
 0.0293101
 0.0168866
 0.0209671
 0.0136236
 0.01504  
 0.0190854
 0.0307193
 0.0251652
 0.0138399
 0.0168999
 0.0275213
 0.0116397
 0.0253615
 0.0338734
 0.0401589
 0.0315266
 0.0137769
 0.0153384
 0.0149432
 0.0160794